In [24]:
import pandas as pd

## Chargment de la base de données test
On récupère les 500 plus grandes entreprises de France (à base d'un copier coller) avant de charger les données SIREN-nomD'entreprise

In [25]:
df = pd.read_csv("../data/top500verifCom.tsv",
                     names=["companyName", "postCode", "city", "CA"], sep="\t")
test_inputs = df[["companyName"]]
print(len(test_inputs))
test_inputs.head()

500


,companyName
0,PSA AUTOMOBILES SA
1,ELECTRICITE DE FRANCE
2,AIRBUS
3,RENAULT SAS
4,ORANGE


In [26]:
siren_table_path = "../data/siren_table.csv"
siren_df = pd.read_csv(siren_table_path)
siren_df.head()

C:\Users\benja\AppData\Local\Temp\ipykernel_27744\2920638919.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  siren_df = pd.read_csv(siren_table_path)


,companyName,Siren
0,TA.MI.RO.SA. HOLDING,802504068
1,M.T.M.,945451169
2,SOCIETE CAPBRETONNAISE DE DISTRIBUTION SOCADI,987020203
3,SOCIETE INTERGRAS,997868807
4,NANTET MENUISERIE,76520279


## Standardization
En plus de l'utilisation des techniques courrantes, de nombreuses itérations ont été effectuées.


In [51]:
import re
def preprocess(companies):
    def prep(company):
        company = company.encode("ascii", errors="ignore").decode()
        company = company.lower()
        chars_to_remove = [")", "(", ".", "|", "[", "]", "{", "}", "'", ",", ";"]
        rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
        company = re.sub(rx, '', company)
        company = re.sub(r'[0-9]+', '', company)
        company = clean_spaces(company)
        return company
    def clean_spaces(text):
        text=text.replace('  ', ' ')
        text=text.strip()
        if len(text) < 1:
            raise Exception("Company Name Too short !!")
        return text

    return companies.apply(prep)

Application du preprocessing aux colonnes des deux tables et matching "un à un", parfait

In [52]:
siren_df["companyNameClean"] = preprocess(siren_df["companyName"])
test_inputs["companyNameClean"] = preprocess(test_inputs["companyName"])

result_df = test_inputs.merge(siren_df, on="companyNameClean", how="left")

missing_joins = result_df["Siren"].isnull().sum()/len(result_df)*100

print(f'{missing_joins:.2f} % missing joins, and {len(result_df)} total joins')

Exception: Company Name Too short !!

In [ ]:
result_df[result_df["Siren"].isnull()]

,companyName_x,companyNameClean,companyName_y,Siren
1,ELECTRICITE DE FRANCE,electricite de france,NaN,NaN
14,TOTALENERGIES MARKETING FRANCE,totalenergies marketing france,NaN,NaN
16,LA FRANCAISE DES JEUX,la francaise des jeux,NaN,NaN
21,SYSTEME U CENTRALE NATIONALE,systeme u centrale nationale,NaN,NaN
24,LA POSTE 75015,la poste,NaN,NaN
...,...,...,...,...
544,TOLL COLLECT GMBH,toll collect gmbh,NaN,NaN
547,DAHER AEROSPACE,daher aerospace,NaN,NaN
559,SOCIETE VITREENNE D'ABATTAGE,societe vitreenne dabattage,NaN,NaN
561,AB INBEV FRANCE,ab inbev france,NaN,NaN


## A étudier plus tard : noms non unique dans chiffre-cle-2020
--> faire à la main pour les 500 plus grosses entreprises
--> attention au merge

Lors de la jointure, des noms d'entreprises correspondent à différents SIREN à droite. En effet, après preprocessing, les noms d'entreprise perdent leur unicité dans siren_table

In [ ]:
g = result_df.groupby("companyNameClean").size()
g.where(g>1).dropna()

companyNameClean
amadeus                                              4.0
as                                                  13.0
autodistribution                                     2.0
axione                                               2.0
bel                                                 13.0
boulanger                                            4.0
chanel                                               3.0
clinea                                               2.0
cora                                                 5.0
csf                                                  4.0
elivia                                               2.0
la halle                                             3.0
orange                                               3.0
orpea                                                2.0
pomona                                               2.0
prosol                                               4.0
roche                                               10.0
sca fruits leg

In [53]:
result_df[result_df.duplicated(subset=["companyNameClean"], keep=False)].sample(20)

,companyName_x,companyNameClean,companyName_y,Siren
100,AS 24,as,AS 24,347538043
360,ROCHE,roche,ROCHE,332186436
89,BOULANGER,boulanger,BOULANGER,494712540
4,ORANGE,orange,ORANGE,879144723
64,CORA,cora,CORA,519084271
355,ROCHE,roche,ROCHE,408739845
407,SYNERGIE,synergie,SYNERGIE,329925010
282,BEL,bel,BEL,513089391
63,CORA,cora,CORA,786920306
283,BEL,bel,BEL,849621776


In [54]:
result_df_2 = result_df.merge(siren_df, left_on="companyName_x", right_on="companyName", suffixes=("_a", "_b"))

In [55]:
result_df_2[result_df_2["Siren_a"]==result_df_2["Siren_b"]]

,companyName_x,companyNameClean_a,companyName_y,Siren_a,companyName,Siren_b,companyNameClean_b
0,AIRBUS,airbus,AIRBUS,383474814,AIRBUS,383474814,airbus
1,ORANGE,orange,ORANGE,879144723,ORANGE,879144723,orange
5,ORANGE,orange,ORANGE,791052525,ORANGE,791052525,orange
9,ORANGE,orange,ORANGE,485070205,ORANGE,485070205,orange
12,CSF,csf,CSF,479228306,CSF,479228306,csf
...,...,...,...,...,...,...,...
766,CASTEL FRERES,castel freres,CASTEL FRERES,482283694,CASTEL FRERES,482283694,castel freres
767,INTERFORUM,interforum,INTERFORUM,612039073,INTERFORUM,612039073,interforum
768,AXIONE,axione,AXIONE,449586544,AXIONE,449586544,axione
771,AXIONE,axione,AXIONE,479787400,AXIONE,479787400,axione


## NLP

In [56]:
def ngrams(string, n=3):
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]
ngrams("benjamin")

['ben', 'enj', 'nja', 'jam', 'ami', 'min']

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
test_inputs = test_inputs[~test_inputs['companyNameClean'].isin(siren_df["companyNameClean"])]
final = pd.concat([test_inputs[['companyNameClean']], siren_df[['companyNameClean']]],
                    ignore_index=True, axis=0)
company_names = final['companyNameClean'].unique().astype('U')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)

In [50]:
company_names[:20]

array(['electricite de france', 'totalenergies marketing france',
       'la francaise des jeux', 'systeme u centrale nationale',
       'la poste    ', 'total raffinage france',
       'esso societe anonyme francaise  ', 'sncf reseau',
       'louis vuitton malletier', 'arcelormittal atlantique et lorraine',
       'global lng', 'totalenergies electricite et gaz france',
       'manuf franc pneumatiq michelin',
       'monoprix exploit par abreviation mpx', 'total se',
       'compagnie dexploitation et de repartition pharmaceutiques de ro',
       'adecco france', 'sodiaal union', 'thevenin ducrot distribution',
       'rexel france'], dtype='<U159')